1. Напишите текстом как минимум 6 функций распределения и соответствующие им функции правдоподобия.
Напишите, какие по вашему мнению целевые переменные из бизнеса (лучше - из страхового) соответствуют этим распределениям (по 2 примера на каждое).
Пример: биномиальное распределение для задачи моделирования пролонгации (продлил клиент полис на следующий период или нет).

1). Нормальное
    - Кредитный рейтинг
    - Доля рынка

2). Равномерное
    - Среднее время обслуживания клиента
    - Среднее время ожидания клиента


3). Биномеальное
    - Уйдет клиент к конкуренту или нет
    - Произайдет страховой случай или нет


4). Экспоненциальное
    - Размер выручки
    - Размер прибыли


5). Пуассона
    - Количество заключенных договоров
    - Количество страховых случаев


6). Стьюдента
    - структура страховых портфелей
    - среднее количество выплат в месяц


2. На основе датасета из ноутбука проведите анализ данных и постройте зависимости частоты и среднего убытка от какого-либо параметра (за исключением уже выполненных в ноутбуке).

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [3]:
df = pd.read_csv('~/Yandex.Disk/geekbrains/Python/MLInBusines/lesson_6/freMPL-R.csv', low_memory=False)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 343080 entries, 0 to 343079
Data columns (total 31 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Exposure           343080 non-null  float64
 1   LicAge             343080 non-null  int64  
 2   RecordBeg          343080 non-null  object 
 3   RecordEnd          181115 non-null  object 
 4   VehAge             177880 non-null  object 
 5   Gender             343080 non-null  object 
 6   MariStat           343080 non-null  object 
 7   SocioCateg         343080 non-null  object 
 8   VehUsage           343080 non-null  object 
 9   DrivAge            343080 non-null  int64  
 10  HasKmLimit         343080 non-null  int64  
 11  BonusMalus         343080 non-null  int64  
 12  VehBody            145780 non-null  object 
 13  VehPrice           145780 non-null  object 
 14  VehEngine          145780 non-null  object 
 15  VehEnergy          145780 non-null  object 
 16  Ve

In [18]:
df['HasKmLimit']

145780    0
145781    0
145782    0
145783    0
145784    0
         ..
310975    0
310976    0
310977    0
310978    0
310979    0
Name: HasKmLimit, Length: 115155, dtype: int64

In [6]:
# Объединяем наборы данных 5-9, удаляем пустые столбцы, удаляем дубликаты

df59 = df.loc[df.Dataset.isin([5, 6, 7, 8, 9])]
df59 = df59.drop(['Dataset'], axis=1)
df59 = df59.dropna(axis=1, how='all')
print('With duplicates\t\t', len(df59))
df59 = df59.drop_duplicates()
print('Without duplicates\t', len(df59))

With duplicates		 165200
Without duplicates	 115155


In [7]:
df = df59

In [9]:
# Вспомогательный столбец для суммирования числа полисов
df['PolicyCount'] = 1
# Вспомогательный столбец для суммирования числа полисов с убытками
df['ClaimCount'] = df['ClaimAmount'] > 0
# Вспомогательный столбец для суммирования числа полисов без убытков
df['NoClaimCount'] = df.PolicyCount - df.ClaimCount
df[['PolicyCount','ClaimAmount', 'ClaimCount', 'NoClaimCount']].head()

,PolicyCount,ClaimAmount,ClaimCount,NoClaimCount
145780,1,0.0,False,1
145781,1,0.0,False,1
145782,1,0.0,False,1
145783,1,0.0,False,1
145784,1,0.0,False,1


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 115155 entries, 145780 to 310979
Data columns (total 23 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Exposure           115155 non-null  float64
 1   LicAge             115155 non-null  int64  
 2   RecordBeg          115155 non-null  object 
 3   RecordEnd          59455 non-null   object 
 4   Gender             115155 non-null  object 
 5   MariStat           115155 non-null  object 
 6   SocioCateg         115155 non-null  object 
 7   VehUsage           115155 non-null  object 
 8   DrivAge            115155 non-null  int64  
 9   HasKmLimit         115155 non-null  int64  
 10  BonusMalus         115155 non-null  int64  
 11  ClaimAmount        115155 non-null  float64
 12  ClaimInd           115155 non-null  int64  
 13  ClaimNbResp        115155 non-null  float64
 14  ClaimNbNonResp     115155 non-null  float64
 15  ClaimNbParking     115155 non-null  float64
 1

In [28]:
param = 'BonusMalus'
df_group_age = df[['Exposure', 'PolicyCount','ClaimAmount', 'ClaimCount', 'NoClaimCount']].groupby(df[param]).sum()
df_group_age = df_group_age.reset_index()
df_group_age['Freq'] = df_group_age.ClaimCount / df_group_age.Exposure
df_group_age['AvgClaim'] = df_group_age.ClaimAmount / df_group_age.ClaimCount
fig = px.bar(df_group_age, x=param, y='Freq', title='Зависимость частоты убытков от коэффициента BonusMalus' )
fig.show()

In [32]:
fig = px.bar(df_group_age, x=param, y='AvgClaim', title='Зависимость среднего убытка от коэффициента BonusMalus')
fig.show()